# COVID-19 Data Cleaning
This notebook fills null values with appropriate values for each column.

In [19]:
import pandas as pd
import pycountry
import pycountry_convert as pc

# Load your dataset
df_origin = pd.read_csv("owid-covid-data.csv")
df = df_origin.copy()
df.isnull().sum()


iso_code                                        0
continent                                   26525
location                                        0
date                                            0
total_cases                                 17631
                                            ...  
population                                      0
excess_mortality_cumulative_absolute       416024
excess_mortality_cumulative                416024
excess_mortality                           416024
excess_mortality_cumulative_per_million    416024
Length: 67, dtype: int64

In [20]:
# Function: get continent name from ISO Alpha-3 code
def get_continent_name(iso_code):
    try:
        country_alpha2 = pc.country_alpha3_to_country_alpha2(iso_code)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        return pc.convert_continent_code_to_continent_name(continent_code)
    except:
        return None

# Function: get country name from ISO Alpha-3 code
def get_country_name(iso_code):
    try:
        country = pycountry.countries.get(alpha_3=iso_code)
        return country.name
    except:
        return None

In [21]:
# Step 1: Fill missing continent using ISO code
df["continent"] = df.apply(
    lambda row: get_continent_name(row["iso_code"]) if pd.isnull(row["continent"]) or row["continent"] in ["0", "", None] else row["continent"],
    axis=1
)

# Step 2: Fill missing location using ISO code
df["location"] = df.apply(
    lambda row: get_country_name(row["iso_code"]) if pd.isnull(row["location"]) or row["location"] == row["continent"] else row["location"],
    axis=1
)

In [22]:
# Step 3: Fix cases where location is a continent name
continent_names = {"Africa", "Asia", "Europe", "North America", "South America", "Oceania", "Antarctica"}
df.loc[df["location"].isin(continent_names), "location"] = df["iso_code"].apply(get_country_name)

# Step 4: Fix cases where continent is a country name
all_country_names = {c.name for c in pycountry.countries}
df.loc[df["continent"].isin(all_country_names), "continent"] = df["iso_code"].apply(get_continent_name)

# Step 5: Final check for any missing values
df["continent"] = df["continent"].fillna(df["iso_code"].apply(get_continent_name))
df["location"] = df["location"].fillna(df["iso_code"].apply(get_country_name))

In [23]:
# Remove OWID special ISO codes
owid_special_iso = ["OWID_WRL", "OWID_AFR", "OWID_ASI","OWID_EUN","OWID_HIC","OWID_LIC","OWID_LMC","OWID_UMC",
                    "OWID_EUR", "OWID_NAM", "OWID_SAM", "OWID_OCE"]
df = df[~df["iso_code"].isin(owid_special_iso)]

In [24]:
df.isnull().sum()

iso_code                                        0
continent                                       0
location                                        0
date                                            0
total_cases                                 11194
                                            ...  
population                                      0
excess_mortality_cumulative_absolute       389499
excess_mortality_cumulative                389499
excess_mortality                           389499
excess_mortality_cumulative_per_million    389499
Length: 67, dtype: int64

In [25]:
columns_to_keep = [
    "iso_code",
    "continent",
    "location",
    "date",
    "total_cases",
    "new_cases",
    "total_deaths",
    "new_deaths",
    "reproduction_rate",
    "total_tests",
    "new_tests",
    "total_vaccinations",
    "people_vaccinated",
    "people_fully_vaccinated",
    "total_boosters",
    "population",
    "population_density",
    "median_age",
    "aged_65_older",
    "aged_70_older",
    "gdp_per_capita",
    "extreme_poverty",
    "hospital_beds_per_thousand",
    "life_expectancy",
    "human_development_index"
]

In [26]:
# Define columns that should not be filled with zero
columns_not_zero = [
    "population", "gdp_per_capita", "life_expectancy", 
    "human_development_index", "median_age", "aged_65_older", 
    "aged_70_older", "extreme_poverty", "hospital_beds_per_thousand"
]

# Fill null values with zero for other columns
for column in df.columns:
    if column not in columns_not_zero:
        df[column] = df[column].fillna(0)

# Check if the filling was successful
df.isnull().sum()
df.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
count,4.029100e+05,4.029100e+05,4.029100e+05,4.029100e+05,402910.000000,402910.000000,402910.000000,402910.000000,402910.00000,402910.000000,...,402910.000000,402910.000000,289005.000000,388615.000000,317443.000000,4.029100e+05,4.029100e+05,402910.000000,402910.000000,402910.000000
mean,1.776320e+06,1.925827e+03,1.925727e+03,1.988027e+04,17.524976,17.523207,109797.372987,119.365142,119.35412,804.922114,...,19.883999,20.080971,3.109266,73.706961,0.722083,3.338239e+07,1.865566e+03,0.325079,0.363654,59.003820
std,7.753085e+06,8.447995e+04,3.188061e+04,8.153673e+04,310.021494,116.048129,162784.071005,1514.507132,561.66312,1131.093238,...,19.459324,31.938947,2.556394,7.403534,0.149627,1.348128e+08,3.033322e+04,2.809710,4.890607,482.874255
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.100000,53.280000,0.390000,4.700000e+01,-3.772610e+04,-44.230000,-95.920000,-2936.450000
25%,3.904000e+03,0.000000e+00,0.000000e+00,2.700000e+01,0.000000,0.000000,1372.800000,0.000000,0.00000,14.140000,...,0.000000,0.000000,1.300000,69.500000,0.600000,3.957620e+05,0.000000e+00,0.000000,0.000000,0.000000
50%,4.517850e+04,0.000000e+00,6.860000e+00,5.325000e+02,0.000000,0.000000,25309.930000,0.000000,1.84000,223.410000,...,18.800000,0.000000,2.500000,75.050000,0.740000,5.466000e+06,0.000000e+00,0.000000,0.000000,0.000000
75%,5.160230e+05,0.000000e+00,1.797100e+02,6.881000e+03,0.000000,1.710000,150786.400000,0.000000,48.95000,1236.730000,...,35.600000,28.180000,4.210000,79.460000,0.830000,2.212524e+07,0.000000e+00,0.000000,0.000000,0.000000
max,1.034368e+08,4.047548e+07,5.782211e+06,1.193165e+06,47687.000000,6812.430000,763598.600000,241758.230000,34536.89000,6601.110000,...,78.100000,100.000000,13.800000,86.750000,0.960000,1.425887e+09,1.349776e+06,78.080000,378.220000,10293.520000


In [27]:
# Fill columns not zero with median by country, fallback to global median
for col in columns_not_zero:
    if df[col].isnull().sum() > 0:
        df[col] = df.groupby('iso_code')[col].transform(lambda x: x.fillna(x.median()))
        df[col] = df[col].fillna(df[col].median())
        
df.describe()

C:\Users\khali\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\khali\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\khali\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\khali\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarnin

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
count,4.029100e+05,4.029100e+05,4.029100e+05,4.029100e+05,402910.000000,402910.000000,402910.000000,402910.000000,402910.00000,402910.000000,...,402910.000000,402910.000000,402910.000000,402910.000000,402910.000000,4.029100e+05,4.029100e+05,402910.000000,402910.000000,402910.000000
mean,1.776320e+06,1.925827e+03,1.925727e+03,1.988027e+04,17.524976,17.523207,109797.372987,119.365142,119.35412,804.922114,...,19.883999,20.080971,2.937023,73.754611,0.725884,3.338239e+07,1.865566e+03,0.325079,0.363654,59.003820
std,7.753085e+06,8.447995e+04,3.188061e+04,8.153673e+04,310.021494,116.048129,162784.071005,1514.507132,561.66312,1131.093238,...,19.459324,31.938947,2.182406,7.275255,0.133014,1.348128e+08,3.033322e+04,2.809710,4.890607,482.874255
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.100000,53.280000,0.390000,4.700000e+01,-3.772610e+04,-44.230000,-95.920000,-2936.450000
25%,3.904000e+03,0.000000e+00,0.000000e+00,2.700000e+01,0.000000,0.000000,1372.800000,0.000000,0.00000,14.140000,...,0.000000,0.000000,1.700000,69.660000,0.650000,3.957620e+05,0.000000e+00,0.000000,0.000000,0.000000
50%,4.517850e+04,0.000000e+00,6.860000e+00,5.325000e+02,0.000000,0.000000,25309.930000,0.000000,1.84000,223.410000,...,18.800000,0.000000,2.500000,75.050000,0.740000,5.466000e+06,0.000000e+00,0.000000,0.000000,0.000000
75%,5.160230e+05,0.000000e+00,1.797100e+02,6.881000e+03,0.000000,1.710000,150786.400000,0.000000,48.95000,1236.730000,...,35.600000,28.180000,3.320000,79.190000,0.810000,2.212524e+07,0.000000e+00,0.000000,0.000000,0.000000
max,1.034368e+08,4.047548e+07,5.782211e+06,1.193165e+06,47687.000000,6812.430000,763598.600000,241758.230000,34536.89000,6601.110000,...,78.100000,100.000000,13.800000,86.750000,0.960000,1.425887e+09,1.349776e+06,78.080000,378.220000,10293.520000


In [28]:
# Columns where negative values don't make sense and should be zeroed out
cols_no_negative = [
    "total_cases", "new_cases", "new_cases_smoothed", "total_deaths", "new_deaths", "new_deaths_smoothed",
    "total_cases_per_million", "new_cases_per_million", "new_cases_smoothed_per_million", "total_deaths_per_million",
    "new_deaths_per_million", "new_deaths_smoothed_per_million", "total_tests", "new_tests",
    "total_vaccinations", "people_vaccinated", "people_fully_vaccinated", "total_boosters"
]

for col in cols_no_negative:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: max(x, 0) if pd.notnull(x) else x)
        


In [29]:
# Keep only the relevant columns
df = df[columns_to_keep]

df.describe()


,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,total_tests,new_tests,total_vaccinations,people_vaccinated,people_fully_vaccinated,...,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,hospital_beds_per_thousand,life_expectancy,human_development_index
count,4.029100e+05,4.029100e+05,4.029100e+05,402910.000000,402910.000000,4.029100e+05,4.029100e+05,4.029100e+05,4.029100e+05,4.029100e+05,...,4.029100e+05,402910.000000,402910.000000,402910.000000,402910.000000,402910.000000,402910.000000,402910.000000,402910.000000,402910.000000
mean,1.776320e+06,1.925827e+03,1.988027e+04,17.524976,0.415250,4.158320e+06,1.259220e+04,1.621726e+07,5.622976e+06,4.851982e+06,...,3.338239e+07,352.342707,30.305818,8.200639,5.191205,17655.871864,8.479913,2.937023,73.754611,0.725884
std,7.753085e+06,8.447995e+04,8.153673e+04,310.021494,0.527967,3.826219e+07,1.103365e+05,1.567674e+08,5.419291e+07,4.852838e+07,...,1.348128e+08,1693.224090,8.293947,5.541662,3.802947,18088.704201,15.643065,2.182406,7.275255,0.133014
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.070000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.700000e+01,0.000000,15.100000,1.140000,0.530000,661.240000,0.100000,0.100000,53.280000,0.390000
25%,3.904000e+03,0.000000e+00,2.700000e+01,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.957620e+05,21.190000,23.300000,4.030000,2.360000,5819.500000,2.200000,1.700000,69.660000,0.650000
50%,4.517850e+04,0.000000e+00,5.325000e+02,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.466000e+06,79.490000,29.600000,6.290000,3.860000,12236.710000,2.500000,2.500000,75.050000,0.740000
75%,5.160230e+05,0.000000e+00,6.881000e+03,0.000000,0.910000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.212524e+07,209.590000,37.300000,10.940000,6.940000,22767.040000,3.300000,3.320000,79.190000,0.810000
max,1.034368e+08,4.047548e+07,1.193165e+06,47687.000000,5.870000,9.214000e+09,3.585563e+07,3.491077e+09,1.310292e+09,1.276760e+09,...,1.425887e+09,20546.770000,48.200000,27.050000,18.490000,116935.600000,77.600000,13.800000,86.750000,0.960000


In [32]:
# Save cleaned file
df.to_csv("covid_data_analysis.csv", index=False)